In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2
import numpy as np
import os
import matplotlib.pyplot as plt

%matplotlib inline
import keras

In [ ]:
#2 Assignment
from keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

In [ ]:
print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))

length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[0]] )
print(decoded) 

In [ ]:
def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
       results[i, sequence] = 1
    return results
 
data = vectorize(data)
targets = np.array(targets).astype("float32")

In [ ]:
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

In [ ]:
from tensorflow.keras.constraints import UnitNorm
from tensorflow.keras.constraints import Constraint
from tensorflow.python.framework import tensor_shape

input_dim = (train_x.shape[1])

encoder = Dense (units = 256 ,activation ="relu",input_shape =( input_dim ,) ,use_bias = True , 
                  kernel_constraint =UnitNorm ( axis =0) ,activity_regularizer =tf.keras.regularizers.L1( l1 =0.01),name ='encoder')
decoder= Dense (units = 1 ,activation ="linear",use_bias = True , kernel_constraint =UnitNorm ( axis =1),name ='decoder')

In [ ]:
model=Sequential()
model.add(encoder)
model.add(decoder)

In [ ]:
model.summary ()
model.compile ( metrics =[ 'accuracy'] ,loss ='mean_squared_error',optimizer ='adam')

In [ ]:
history =  model. fit (x = train_x ,y = train_y ,batch_size =32 ,epochs =10 ,validation_data =( test_x ,test_y ) ,verbose =0).history

In [ ]:
encoder_model =tf.keras.Model ( inputs = model . inputs ,outputs = model. get_layer ('encoder'). output )
X_train_encoded_features = encoder_model . predict ( train_x )
#X_valid_encoded_features = encoder_model . predict ( test_x )
X_test_encoded_features =encoder_model . predict ( test_x )

 # Model
classifier = Sequential ()
classifier . add ( tf.keras.Input (shape =( X_train_encoded_features . shape [1] , ) ) )
classifier . add ( Dense ( units =32 ,activation ='relu') )
classifier . add ( Dense ( units =16 ,activation ='relu') )
classifier . add ( Dense ( units =1 ,activation ='sigmoid') )
classifier . compile ( optimizer ='adam',loss ='binary_crossentropy',metrics =[ 'accuracy' ])


In [ ]:
history = classifier . fit (x = X_train_encoded_features ,y = train_y ,batch_size =128 ,epochs =10 ,validation_data =( X_valid_encoded_features ,test_y ) ,verbose =0)

In [ ]:
#list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()